In [1]:
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import os
import torch
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights
import torch.optim as optim
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
# problema librerie che usano OpenMP, due versioni di OpenMP sono caricate nello stesso processo, ad esempio da PyTorch e da NumPy

In [2]:
dataset_path = os.path.join(os.getcwd(), "DAML_project", 'data_images')
print(dataset_path)

/home/user/DAML/DAML_project/data_images


In [3]:
# Mean e std per immagini RGB normalizzate su [-1, 1]
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# per fare data augmentation utiliziammo transforms.Compose di torchvision, facciamo data augmentation solo ai set di train

In [4]:
train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, "train"), transform=train_transform)
val_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, "valid"), transform=val_test_transform)
test_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, "test"), transform=val_test_transform)


print(f"Train dataset size: {len(train_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Classi leggibili (opzionale)
pretty_classes = ['Adenocarcinoma', 'Adgelcarcinoma', 'Squamosgelcarcinoma', 'Noncancer']

Train dataset size: 613


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet18(weights=ResNet18_Weights.DEFAULT) # trasnfer learning (pre-trained model)

# congela i layer convoluzionali per fare fine-tuning solo sull'ultimo layer
for param in model.parameters():
    param.requires_grad = False

# Sblocca solo l'ultimo FC Layer
model.fc = nn.Identity()  # rimuove l'ultimo layer FC
model.eval()

for inputs, labels in train_loader:
    # optimizer.zero_grad()
    outputs = model(inputs)
    # loss = criterion(outputs, labels)
    # loss.backward()
    # optimizer.step()
    print(outputs.shape)
    break


torch.Size([613, 512])
